In [1]:
import pandas as pd
import numpy as np

In [74]:
data = pd.read_csv('amazon_alexa.tsv',delimiter='\t')
data.head()

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [75]:
data['rating_label'] = data['rating'].apply(lambda r:'pos' if r > 3 else 'neg')
data.drop(['date','variation','feedback','rating'],axis=1,inplace=True)
data.head()

,verified_reviews,rating_label
0,Love my Echo!,pos
1,Loved it!,pos
2,"Sometimes while playing a game, you can answer...",pos
3,I have had a lot of fun with this thing. My 4 ...,pos
4,Music,pos


In [76]:
data.isnull().sum()

verified_reviews    0
rating_label        0
dtype: int64

In [77]:
blanks = []  # start with an empty list
for i,lab,re in data.itertuples():
    if type(re)==str:            # avoid NaN values
        if re.isspace():         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list
print(len(blanks))

0


In [78]:
data['rating_label'].value_counts()

pos    2741
neg     409
Name: rating_label, dtype: int64

In [79]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [85]:
data['scores'] = data['verified_reviews'].apply(lambda review : sid.polarity_scores(review))
data['compound'] = data['scores'].apply(lambda score_dict: score_dict['compound'])
data['comp_score'] = data['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')
data.head()

,verified_reviews,rating_label,scores,compound,comp_score
0,Love my Echo!,pos,"{'neg': 0.0, 'neu': 0.308, 'pos': 0.692, 'comp...",0.6696,pos
1,Loved it!,pos,"{'neg': 0.0, 'neu': 0.193, 'pos': 0.807, 'comp...",0.6360,pos
2,"Sometimes while playing a game, you can answer...",pos,"{'neg': 0.102, 'neu': 0.784, 'pos': 0.114, 'co...",-0.1280,neg
3,I have had a lot of fun with this thing. My 4 ...,pos,"{'neg': 0.0, 'neu': 0.617, 'pos': 0.383, 'comp...",0.9169,pos
4,Music,pos,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos


In [91]:
data.iloc[2,0]

'Sometimes while playing a game, you can answer a question correctly but Alexa says you got it wrong and answers the same as you.  I like being able to turn lights on and off while away from home.'

In [92]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
accuracy_score(data['rating_label'],data['comp_score'])

0.8853968253968254

In [93]:
print(classification_report(data['rating_label'],data['comp_score']))

              precision    recall  f1-score   support

         neg       0.61      0.34      0.43       409
         pos       0.91      0.97      0.94      2741

   micro avg       0.89      0.89      0.89      3150
   macro avg       0.76      0.65      0.68      3150
weighted avg       0.87      0.89      0.87      3150



In [94]:
print(confusion_matrix(data['rating_label'],data['comp_score']))

[[ 138  271]
 [  90 2651]]


In [97]:
data[data['comp_score'] == 'neg'].iloc[5,0]

'I like being able to tell Alexia the music I want to hear, but sometimes she has trouble finding the song if I am not very specific about the title.'